<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Coherence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install lime
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283859 sha256=24aff90e472b8aeb0ccc924f9435701fda0608a4de14b8cb2b458c48243a1387
  Stored in directory: /root/.cache/pip/wheels/ed/d7/c9/5a0130d06d6310bc6cbe55220e6e72dcb8c4eff9a478717066
Successfully built lime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 32.

In [ ]:
import torch
from lime.lime_text import LimeTextExplainer
from transformers import pipeline, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import pandas as pd
import shap
import scipy as sp
import numpy as np
import gensim
from gensim.models import LdaModel
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim.models import CoherenceModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
!nvidia-smi -L    

GPU 0: Tesla T4 (UUID: GPU-289a28d4-dddc-64a0-6c84-e91c91563692)


In [ ]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)

In [ ]:
train = pd.read_csv(r'/content/drive/MyDrive/data.csv')

In [ ]:
train

,Unnamed: 0.1,Unnamed: 0,Text,Label
0,0,0,मैं अपने लिए तो अपने आप बनाती हूँ |,2
1,1,1,और वो लोग अपने लिए अपने आप बनाते है उनकी फॅमिली |,2
2,2,2,में अलग हो गया था अलग हो गया था सब मतलब 1 साल ...,1
3,3,3,नहीं वो है बड़ी वाली तो जो उनकी बेटी है और जो म...,0
4,4,4,मै मेरको ना (english : actually time ) लगता था...,1
...,...,...,...,...
6494,6494,7209,"हे ना , वो इतना पढ़ी - लिखी थी वो मेरे से बेटर...",1
6495,6495,7210,"मेरे ये था में मतलब मुझे , अभी भी ऐसा हो रहा ह...",1
6496,6496,7211,जैसे मुझे कोई चीज़ अगर ऐसी वैसी लगती है कोई भी ...,1
6497,6497,7212,"हाँ, उसके बाद में ही मुझे सेटिस्फैक्शन फील ( e...",1


In [ ]:
test = pd.read_csv(r'/content/drive/MyDrive/test.csv')

In [ ]:
test_sent = test['hindi_text'][:]

## Make Corpus and define Topics to measure Coherence

In [ ]:
documents = train['Text'][:].tolist()
# Create a dictionary from your text data
dataset = [d.split() for d in documents]
dictionary = corpora.Dictionary(dataset)

# Create a corpus from your dictionary and text data
corpus = [dictionary.doc2bow(doc) for doc in dataset]

In [ ]:
# Train an LDA model on your corpus
num_topics = 10  # Change this to the number of topics you want to extract
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

In [ ]:
def coherence_score(important_words):
  # Compute the coherence score using the list of important words and the LDA model
  coherence_model_lda = CoherenceModel(model=lda_model, texts=[important_words], dictionary=dictionary, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  return coherence_lda

##LIME

In [ ]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  #predicted_class_id = logits.argmax().item()
  # classifications = ['Introduction','Depression','Grey Area']
  # classification = logits.argmax().item()
  # class_name = classifications[classification]
  
  return scores

def pre_process(data):
  inputs = tokenizer(data, 
            return_tensors='pt', 
            padding=True, 
            truncation=True, 
            max_length=128)
  return inputs

def predictor(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  predicted_class_id = logits.argmax().item()
  classifications = ['Introduction','Depression','Grey Area']
  classification = logits.argmax().item()
  class_name = classifications[classification]
  
  return class_name


In [ ]:
class_name = ['Introduction','Depression','Grey Area']
explainer = LimeTextExplainer(class_names = class_name)

In [ ]:
# Use LDA to evaluate the quality of the LIME explanations
vectorizer = CountVectorizer( max_df=5, min_df=0.01)
lda = LatentDirichletAllocation(n_components=5, max_iter=5, learning_method='online', random_state=42)

In [ ]:
def get_score(text):
  exp = explainer.explain_instance(text,adapter)
  exp_words = [word for word, weight in exp.as_list()]
  X = vectorizer.fit_transform(exp_words)
  lda.fit(X)
  return lda.score(X)

In [ ]:
lime_score = []

In [ ]:
for text in test_sent:
  lime_score.append(get_score(text))

## SHAP

In [ ]:
## SHAP adaptor and predictor
def adapter_shap(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  predicted_class_id = logits.argmax().item()
  classifications = ['Introduction','Depression','Grey Area']
  classification = logits.argmax().item()
  class_name = classifications[classification]
  return class_name

def predictor_shap(x):
    probas = model(**tokenizer(x, return_tensors="pt", padding=True))[0].softmax(1).detach().cpu().numpy()
    # logits = model(**tokenizer(x, return_tensors="pt", padding=True)).logits
    # predicted_class_id = logits.argmax().item()
    # classifications = ['Introduction','Depression','Grey Area']
    # classification = logits.argmax().item()
    # class_name = classifications[classification]
    # print(class_name)
    s = sp.special.logit(probas[:,1])
    return s

def f_batch(x):
    val = np.array([])
    for i in x:
      val = np.append(val, predictor_shap(i))
    return val
     

In [ ]:
explainer = shap.Explainer(f_batch, tokenizer)
def get_score(text):
  data = {}
  test = { 'text': [text]}
  shap_values = explainer(test)
  # Get the SHAP values and original text features for the current example
  values = shap_values[0].values
  features = shap_values[0].data
  X = vectorizer.fit_transform(features)
  lda.fit(X)
  return lda.score(X)

In [ ]:
shap_score = []
for text in test_sent:
  shap_score.append(get_score(text))

  0%|          | 0/306 [00:00<?, ?it/s]

Partition explainer: 2it [00:14, 14.31s/it]               


  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 2it [00:10, 10.02s/it]               


...

Partition explainer: 2it [00:10, 10.31s/it]               


  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 2it [00:10, 10.26s/it]               


  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/306 [00:00<?, ?it/s]

In [ ]:
shap_score

[-16.997473882088535,
 -17.00746187586165,
 -11.143767669611378,
 -53.22215135189596,
 -17.007457332933257,
 -13.21092138995191,
 -23.49137737573114,
 -28.167857689098135,
 -28.167854235500467,
 -6.141177461458026,
 -44.32503406191286,
 -44.114013611416695,
 -44.0619217141709,
 -28.203422671867127,
 -11.143768000258003,
 -6.141190246994194,
 -11.143776343912501,
 -2.0274921112109037,
 -7.995668192807836,
 -44.71237004693055,
 -29.963953893791114,
 -22.82070460202841,
 -16.782218570032896,
 -27.473644330530664,
 -57.06159295293936,
 -22.846065240472292,
 -16.552740368145386,
 -2.0274921112109037,
 -11.143771109006662,
 -11.143777446948608,
 -34.41583550125856,
 -16.571290243341682,
 -33.785511880322986,
 -17.00747290734047,
 -11.143775954883454,
 -43.34689511980696,
 -2.0274921112109037,
 -53.15197406727151,
 -25.541612238488593,
 -17.007451637248238,
 -17.068721329813503,
 -46.053858595465364,
 -50.37718272035136,
 -11.143772999356203,
 -28.167836119928957,
 -2.0274921112109037,
 -17.0